In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split


In [2]:
import joblib
import pandas as pd

data_ki = joblib.load('/Users/pinchichen/2025S lab/AI drug project/Catpred/dataset/ki_with_features.joblib')
data_ki.head()

,Sequence,sequence_source,uniprot,Smiles,value,ec,log10_value,log10ki_mean,metabolite_features,protein_features
0,MHTKGPTPQQHDGSALRIGIVHARWNETIIEPLLAGTKAKLLACGV...,uniprot_search,A0A6P8BCR9,O=c1n(C[C@H](O)[C@H](O)[C@H](O)CO)c2nc(O)nc(O)...,0.000002,2.5.1.78,-5.698970,-5.698970,"[0.17249086, -0.23665705, -0.11737049, -0.0940...","[0.026722027, -0.06119407, 0.041440543, -0.109..."
1,MASPDWGYDDKNGPEQWSKLYPIANGNNQSPVDIKTSETKHDTSLK...,brenda,P00915,CN1CCN(C(=O)c2ccc(S(N)(=O)=O)cc2)[C@@H](Cc2ccc...,0.000707,4.2.1.1,-3.150581,-3.150581,"[0.4240491, 0.28231123, 0.2642891, -0.7667821,...","[-0.029181704, -0.12635791, 0.10866662, 0.0339..."
2,MLDDIRGFMNTFSETMFMDVINYSLTRDRYDSVFLQRQNYRDLGQL...,brenda,Q9HLD0,C[C@H]1O[C@H](O[C@H]2[C@H](O)[C@@H](O)[C@@H](O...,0.002990,3.2.1.20,-2.524329,-2.524329,"[0.50005734, 0.3923428, 0.064121775, -0.635894...","[0.02874529, -0.17312695, -0.0026887748, 0.035..."
3,MKLMENIFDLAKANKKKIVLAEGEEERNIRASEEIIKDGIADIILV...,uniprot_search,A0A1L5F6Z9,CCN=C(O)CCN=C(O)[C@H](O)C(C)(C)COP(=O)(O)OP(=O...,0.020000,2.3.1.8,-1.698970,-1.698970,"[0.28353548, 0.018489605, -0.5459905, -0.34693...","[-0.06555556, 0.04763434, 0.031432115, 0.17143..."
4,MAKRIVEPFRIKMVEKIRVPSREEREAALKEAGYNPFLLPSSAVYI...,sabio,P28796,CC(N)C(=O)O,24.500000,4.1.99.1,1.389166,1.419631,"[0.051456872, 0.35619217, 0.10593035, -1.04098...","[0.049294557, -0.043596543, 0.0546125, 0.35600..."


In [3]:
data_ki.rename(columns={'log10ki_mean':'label'},inplace=True)
data_ki.head()

,Sequence,sequence_source,uniprot,Smiles,value,ec,log10_value,label,metabolite_features,protein_features
0,MHTKGPTPQQHDGSALRIGIVHARWNETIIEPLLAGTKAKLLACGV...,uniprot_search,A0A6P8BCR9,O=c1n(C[C@H](O)[C@H](O)[C@H](O)CO)c2nc(O)nc(O)...,0.000002,2.5.1.78,-5.698970,-5.698970,"[0.17249086, -0.23665705, -0.11737049, -0.0940...","[0.026722027, -0.06119407, 0.041440543, -0.109..."
1,MASPDWGYDDKNGPEQWSKLYPIANGNNQSPVDIKTSETKHDTSLK...,brenda,P00915,CN1CCN(C(=O)c2ccc(S(N)(=O)=O)cc2)[C@@H](Cc2ccc...,0.000707,4.2.1.1,-3.150581,-3.150581,"[0.4240491, 0.28231123, 0.2642891, -0.7667821,...","[-0.029181704, -0.12635791, 0.10866662, 0.0339..."
2,MLDDIRGFMNTFSETMFMDVINYSLTRDRYDSVFLQRQNYRDLGQL...,brenda,Q9HLD0,C[C@H]1O[C@H](O[C@H]2[C@H](O)[C@@H](O)[C@@H](O...,0.002990,3.2.1.20,-2.524329,-2.524329,"[0.50005734, 0.3923428, 0.064121775, -0.635894...","[0.02874529, -0.17312695, -0.0026887748, 0.035..."
3,MKLMENIFDLAKANKKKIVLAEGEEERNIRASEEIIKDGIADIILV...,uniprot_search,A0A1L5F6Z9,CCN=C(O)CCN=C(O)[C@H](O)C(C)(C)COP(=O)(O)OP(=O...,0.020000,2.3.1.8,-1.698970,-1.698970,"[0.28353548, 0.018489605, -0.5459905, -0.34693...","[-0.06555556, 0.04763434, 0.031432115, 0.17143..."
4,MAKRIVEPFRIKMVEKIRVPSREEREAALKEAGYNPFLLPSSAVYI...,sabio,P28796,CC(N)C(=O)O,24.500000,4.1.99.1,1.389166,1.419631,"[0.051456872, 0.35619217, 0.10593035, -1.04098...","[0.049294557, -0.043596543, 0.0546125, 0.35600..."


In [4]:
# Define dataset
class MPI_Dataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        return {
            'metabolite_features': torch.tensor(np.asarray(row['metabolite_features'], dtype=np.float32)),
            'protein_features': torch.tensor(np.asarray(row['protein_features'], dtype=np.float32)),
            'label': torch.tensor(float(row['label']), dtype=torch.float32),
        }

In [5]:
# Separate the dataset by unique proteins and drugs
unique_proteins = data_ki['Sequence'].unique()
unique_mols = data_ki['Smiles'].unique()
# Set the seed for reproducibility
torch.manual_seed(42)
# Function to perform a cold split
def cold_split(unique_items, test_size=0.2, val_size=0.1):
    train_items, test_items = train_test_split(unique_items, test_size=test_size, random_state=42)
    train_items, val_items = train_test_split(train_items, test_size=val_size / (1 - test_size), random_state=42)
    return train_items, val_items, test_items
# Cold split by proteins
train_proteins, val_proteins, test_proteins = cold_split(unique_proteins)
train_cold_protein = data_ki[data_ki['Sequence'].isin(train_proteins)]
val_cold_protein = data_ki[data_ki['Sequence'].isin(val_proteins)]
test_cold_protein = data_ki[data_ki['Sequence'].isin(test_proteins)]
# Cold split by molecules
train_mols, val_mols, test_mols = cold_split(unique_mols)
train_cold_mols = data_ki[data_ki['Smiles'].isin(train_mols)]
val_cold_mols = data_ki[data_ki['Smiles'].isin(val_mols)]
test_cold_mols = data_ki[data_ki['Smiles'].isin(test_mols)]

In [6]:
def df2array(df):
    X = np.array([
    np.concatenate([m, p])
    for m, p in zip(df['metabolite_features'], df['protein_features'])])
    y = df['label']
    return X, y

train_X, train_y = df2array(train_cold_protein)
val_X, val_y = df2array(val_cold_protein)
test_X, test_y = df2array(test_cold_protein)

train_X_mols, train_y_mols = df2array(train_cold_mols)
val_X_mols, val_y_mols = df2array(val_cold_mols)
test_X_mols, test_y_mols = df2array(test_cold_mols)


In [7]:
#model initialization
import catboost as cat
ca = cat.CatBoostRegressor()
ca_2 = cat.CatBoostRegressor()

In [8]:
'''
#Grid search
#ref: https://xgboost.readthedocs.io/en/stable/parameter.html
from sklearn.model_selection import GridSearchCV
param_grid = dict(
    max_depth = [5,10,15,20,None],
    n_estimators = [50,100,150,200]
)
regressor = GridSearchCV(estimator = xgb, param_grid = param_grid, cv = pds, verbose = 3, n_jobs = -1)
'''
#Train the model
ca.fit(train_X,train_y)
ca_2.fit(train_X_mols,train_y_mols)

Learning rate set to 0.057171
0:	learn: 1.9882354	total: 105ms	remaining: 1m 44s
1:	learn: 1.9584500	total: 128ms	remaining: 1m 3s
2:	learn: 1.9332547	total: 158ms	remaining: 52.4s
3:	learn: 1.9067204	total: 186ms	remaining: 46.2s
4:	learn: 1.8818956	total: 217ms	remaining: 43.1s
5:	learn: 1.8591431	total: 243ms	remaining: 40.3s
6:	learn: 1.8391950	total: 321ms	remaining: 45.5s
7:	learn: 1.8189458	total: 353ms	remaining: 43.7s
8:	learn: 1.8011051	total: 379ms	remaining: 41.8s
9:	learn: 1.7842654	total: 402ms	remaining: 39.8s
10:	learn: 1.7684098	total: 430ms	remaining: 38.6s
11:	learn: 1.7530894	total: 463ms	remaining: 38.1s
12:	learn: 1.7392305	total: 489ms	remaining: 37.2s
13:	learn: 1.7270223	total: 517ms	remaining: 36.4s
14:	learn: 1.7141616	total: 546ms	remaining: 35.8s
15:	learn: 1.7028348	total: 567ms	remaining: 34.9s
16:	learn: 1.6921168	total: 594ms	remaining: 34.4s
17:	learn: 1.6819269	total: 623ms	remaining: 34s
18:	learn: 1.6726568	total: 650ms	remaining: 33.6s
19:	learn: 1

In [9]:
# Train
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, median_absolute_error, explained_variance_score
from scipy.stats import pearsonr

def metrics_cal(model,X,y):
    y_pred = model.predict(X)
    mse = mean_squared_error(y, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y, y_pred)
    r2 = r2_score(y, y_pred)
    pearson_corr, _ = pearsonr(y, y_pred)
    median_ae = median_absolute_error(y, y_pred)
    explained_var = explained_variance_score(y, y_pred)
    return mse, rmse, mae, r2, pearson_corr, median_ae, explained_var


train_mse, train_rmse, train_mae, train_r2, train_pearson_corr, train_median_ae, train_explained_var = metrics_cal(ca, train_X, train_y)
val_mse, val_rmse, val_mae, val_r2, val_pearson_corr, val_median_ae, val_explained_var = metrics_cal(ca, val_X, val_y)
test_mse, test_rmse, test_mae, test_r2, test_pearson_corr, test_median_ae, test_explained_var = metrics_cal(ca, test_X, test_y)


In [10]:
train_mse_mols, train_rmse_mols, train_mae_mols, train_r2_mols, train_pearson_corr_mols, train_median_ae_mols, train_explained_var_mols = metrics_cal(ca_2, train_X_mols, train_y_mols)
val_mse_mols, val_rmse_mols, val_mae_mols, val_r2_mols, val_pearson_corr_mols, val_median_ae_mols, val_explained_var_mols = metrics_cal(ca_2, val_X_mols, val_y_mols)
test_mse_mols, test_rmse_mols, test_mae_mols, test_r2_mols, test_pearson_corr_mols, test_median_ae_mols, test_explained_var_mols = metrics_cal(ca_2, test_X_mols, test_y_mols)

In [11]:
# append the performance to the csv file
df = {
    'Model':['CatBoost','CatBoost','CatBoost'],
    'Dataset':['Train','Validation','Test'],
    'MSE':[train_mse,val_mse,test_mse],
    'RMSE':[train_rmse,val_rmse,test_rmse],
    'MAE':[train_mae,val_mae,test_mae],
    'R2':[train_r2,val_r2,test_r2],
    'Pearson':[train_pearson_corr,val_pearson_corr,test_pearson_corr],
    'Median_AE':[train_median_ae,val_median_ae,test_median_ae],
    'Explained_VAR':[train_explained_var,val_explained_var,test_explained_var],
    'Dataspliting Mode':['cold protein','cold protein','cold protein']
}
df = pd.DataFrame(df)

df.to_csv('/Users/pinchichen/2025S lab/AI drug project/Catpred/Ki/model performance metrics_Catpred_Ki.csv', mode='a', header=False)

In [12]:
# append the performance to the csv file
df = {
    'Model':['CatBoost','CatBoost','CatBoost'],
    'Dataset':['Train','Validation','Test'],
    'MSE':[train_mse_mols,val_mse_mols,test_mse_mols],
    'RMSE':[train_rmse_mols,val_rmse_mols,test_rmse_mols],
    'MAE':[train_mae_mols,val_mae_mols,test_mae_mols],
    'R2':[train_r2_mols,val_r2_mols,test_r2_mols],
    'Pearson':[train_pearson_corr_mols,val_pearson_corr_mols,test_pearson_corr_mols],
    'Median_AE':[train_median_ae_mols,val_median_ae_mols,test_median_ae_mols],
    'Explained_VAR':[train_explained_var_mols,val_explained_var_mols,test_explained_var_mols],
    'Dataspliting Mode':['cold mols','cold mols','cold mols']
}
df = pd.DataFrame(df)

df.to_csv('/Users/pinchichen/2025S lab/AI drug project/Catpred/Ki/model performance metrics_Catpred_Ki.csv', mode='a', header=False)

In [13]:
# save the model
ca.save_model("/Users/pinchichen/2025S lab/AI drug project/Catpred/Ki/trained_model/cold_protein/CatBoost model_Catpred_Ki_cold_protein.json", format="json")
ca_2.save_model("/Users/pinchichen/2025S lab/AI drug project/Catpred/Ki/trained_model/cold_mols/CatBoost model_Catpred_Ki_cold_mols.json",format='json')

# load the model
#model = CatBoostRegressor()
#model.load_model("catboost_model.cbm"